# Setup and Run Batchlinks


In [ ]:
#@title Install and run

!pip install gradio==3.16.2
!git clone https://github.com/etherealxx/batchlinks-webui /content/sdw/extensions/batchlinks-webui
!python /content/sdw/extensions/batchlinks-webui/scripts/batchlinks-downloader.py

In [ ]:
#@title Run

!python /content/sdw/extensions/batchlinks-webui/scripts/batchlinks-downloader.py

# Uploading to HuggingFace

Cells ~copied~ forked from [NoCrypt's Colab Remastered](https://colab.research.google.com/drive/1wEa-tS10h4LlDykd87TF5zzpXIIQoCmq#scrollTo=EzbWAsEc2bAt)

In [ ]:
#@title 1. Login to Huggingface hub
try:
  hub_ok
except:
  print("Setting up huggingface_hub...")
  !pip install --force-reinstall -qqqq huggingface_hub
  hub_ok = True
from IPython.display import clear_output
from huggingface_hub import login
clear_output()

#@markdown 1. Of course, you need a Huggingface account first.
#@markdown 2. To create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.

write_token = "" #@param {type:"string"}
login(write_token, add_to_git_credential=True)

In [ ]:
#@title 2. Setup Repo
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi

api = HfApi()
user = api.whoami(write_token)

#@markdown #### If your model repo didn't exist, it will automatically create your repo.
repo_name = "my-model-repo" #@param{type:"string"}
make_this_repo_private_if_not_exist = True #@param{type:"boolean"}

if "/" in repo_name:
  model_repo = repo_name.strip()
else:
  model_repo = user['name']+"/"+repo_name.strip()

validate_repo_id(model_repo)

if repo_name != "":
  try:
      api.create_repo(repo_id=model_repo, 
                      private=make_this_repo_private_if_not_exist)
      print("Model Repo didn't exists, creating repo")
      print("Model Repo: ",model_repo,"created!\n")

  except HfHubHTTPError as e:
      print(f"Model Repo: {model_repo} exists, skipping create repo\n")

In [ ]:
#@title 3.2 Upload via huggingface_hub (Manual way)

#@markdown All paths for easier access:<br>
#@markdown /content/sdw/models/Stable-diffusion<br>
#@markdown /content/sdw/models/VAE<br>
#@markdown /content/sdw/models/Lora<br>
#@markdown /content/sdw/embeddings<br>
#@markdown /content/sdw/models/hypernetwork<br>

from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
file_path = "/content/my-awesome-model.safetensors" #@param {type :"string"}
#@markdown Leave this blank if you just want to upload it to the repo, dont leave a trailing "`/`" at the end!
path_in_repo = "" #@param {type :"string"}
#@markdown Leaving this blank auto fill the commit message with a generic one
commit_message = "" #@param {type :"string"}

if file_path != "":
  path_obj = Path(file_path)
  model = path_obj.parts[-1]

  if path_in_repo == "":
    trained_model = model
  else: 
    trained_model = path_in_repo.strip() + "/" + model

  print(f"Uploading {model} to https://huggingface.co/"+model_repo)
  print(f"Please wait...")

  if commit_message == "":
    commit_message = f"Upload {model}"

  api.upload_file(
      path_or_fileobj=file_path,
      path_in_repo=trained_model,
      repo_id=model_repo,
      commit_message=commit_message,
  )
  
  print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")